In [65]:
from pyspark.sql import functions as F
from pyspark.sql import types
from pyspark.sql import Row

from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import GradientBoostedTrees
from pyspark.mllib.tree import RandomForest
from pyspark.mllib.classification import SVMWithSGD
from pyspark.mllib.classification import LogisticRegressionWithSGD
from pyspark.mllib.linalg.distributed import IndexedRow, IndexedRowMatrix
from pyspark.mllib.linalg import Vectors, DenseVector, SparseVector
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

import pandas as pd
import numpy as np
from IPython.display import display

from collections import defaultdict
from sklearn import metrics

In [3]:
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [6]:
contest_data_sample = sqlContext.sql("select * from ignite.contest_data_sample")

In [7]:
contest_so_end_customer = sqlContext.sql("select * from edso_ignite.contest_so_end_customer")

contest_sol_end_customer = sqlContext.sql("select * from edso_ignite.contest_sol_end_customer")

contest_address_info = sqlContext.sql("select * from edso_ignite.contest_address_info")

contest_party_info = sqlContext.sql("select * from edso_ignite.contest_party_info")

contest_cr = sqlContext.sql("select * from edso_ignite.contest_cr")

contest_savm = sqlContext.sql("select * from edso_ignite.contest_savm").repartition(200).cache()

contest_end_customer = sqlContext.sql("select * from edso_ignite.contest_end_customer")

In [5]:
sqlContext.sql("set spark.sql.shuffle.partitions=20")

DataFrame[key: string, value: string]

In [18]:
contest_data = sqlContext.sql("select * from edso_ignite.contest_data")
contest_data.withColumn('sales_acct_id', contest_data.sales_acct_id.cast(types.DecimalType(10, 1)))


DataFrame[endcustomerlinefixed: string, pppk: int, fppk: int, decision_date_time: string, days_from_create_to_final_decision: double, end_customer_party_ssot_party_id_int_sav_party_id: int, prior_party_ssot_party_id_int_sav_party_id: int, final_party_ssot_party_id_int_sav_party_id: int, type: string, priorpartyname: string, finalpartyname: string, sales_order_key: int, sales_order_line_key: int, so_number_int: int, end_customer_key_line_level: int, end_customer_key_order_level: int, dd_end_customer_type_order_level: string, purchase_order_type_code: string, purchase_order_number: string, order_datetime: string, oracle_book_datetime: string, transactional_currency_code: string, customer_service_rep_name: string, cust_svc_rep_team_name: string, conversion_type_code: string, original_system_reference_code: string, submitted_on_web_by_name: string, deal_id: string, price_list_name: string, so_source_name: string, order_type_name: string, so_src_crt_datetime: string, sales_channel_code: str

In [8]:
check_skewness(builder)

NameError: name 'builder' is not defined

In [9]:
#contest_data_sample = contest_data

In [7]:
#contest_data_sample.write.saveAsTable("ignite.contest_data_sample", mode = 'overwrite')

In [6]:
contest_data_r = contest_data_sample.select(*(F.col(x).alias(x + '_data') for x in contest_data_sample.columns))
contest_savm_r = contest_savm.select(*(F.col(x).alias(x + '_savm') for x in contest_savm.columns))

builder = contest_data_r.join(contest_savm_r, contest_data_r.endcustomerlinefixed_data == contest_savm_r.party_name_savm, how = 'inner')

In [7]:
contest_data_sample.count()

1057

In [8]:
#groupby and count, save as dataframe
grpd = builder.groupby('endcustomerlinefixed_data','party_name_savm','sales_acct_id_savm').count().orderBy(["endcustomerlinefixed_data", "count"], ascending=[0, 0]).dropDuplicates(['endcustomerlinefixed_data']).drop('count')

In [9]:
grpd_r = grpd.select(*(F.col(x).alias(x + '_grpd') for x in grpd.columns))

result = contest_data_r.join(grpd_r, contest_data_r.endcustomerlinefixed_data == grpd_r.endcustomerlinefixed_data_grpd, how = 'left').select('endcustomerlinefixed_data','party_name_savm_grpd','sales_acct_id_savm_grpd','sales_acct_id_data')

In [77]:
def find_prediction(savm_id,data_id):
    return 1 if savm_id==data_id else 0

find_prediction_udf = F.udf(find_prediction,types.IntegerType())
result = result.withColumn("predicted", find_prediction_udf(result["sales_acct_id_savm_grpd"],result["sales_acct_id_data"]))#.select(['endcustomerlinefixed_data','predicted'])

In [28]:
contest_data_unq = contest_data_sample.select('endcustomerlinefixed').distinct()#.count()#5
builder_unq = builder.select('endcustomerlinefixed_data').distinct()#.count()#5
grpd_unq = grpd.select('endcustomerlinefixed_data').distinct()#.count()#5
result_unq = result.select('endcustomerlinefixed_data').distinct()#.count()#5
#contest_data_unq.show(60,False)
print(contest_data_unq.count(),result_unq.count())

(578, 578)


In [14]:
result.where(F.col('predicted')==1).count()

755

In [15]:
builder_unq.count()

505

In [80]:
result.where((F.col('party_name_savm_grpd').isNull())).count()

99

In [78]:
result.where((~F.col('party_name_savm_grpd').isNull()) & (F.col('predicted')==0)).select('endcustomerlinefixed_data','party_name_savm_grpd','predicted').count()

196

In [81]:
result.where((~F.col('party_name_savm_grpd').isNull()) & (F.col('predicted')==0)).select('endcustomerlinefixed_data').distinct().count()

119

In [47]:
result.where((~F.col('party_name_savm_grpd').isNull()) & (F.col('predicted')==0)).select('endcustomerlinefixed_data','party_name_savm_grpd','predicted').sample(False, 0.1).show(100,False)

+--------------------------------------------+--------------------------------------------+---------+
|endcustomerlinefixed_data                   |party_name_savm_grpd                        |predicted|
+--------------------------------------------+--------------------------------------------+---------+
|BECHTEL MARINE PROPULSION CORPORATION       |BECHTEL MARINE PROPULSION CORPORATION       |0        |
|BECHTEL MARINE PROPULSION CORPORATION       |BECHTEL MARINE PROPULSION CORPORATION       |0        |
|HEALTH AND HUMAN SERVICES AGENCY DATA CENTER|HEALTH AND HUMAN SERVICES AGENCY DATA CENTER|0        |
|NAVAL DISTRICT WASHINGTON                   |NAVAL DISTRICT WASHINGTON                   |0        |
|CSCTEXTRON-CESSNA                           |CSCTEXTRON-CESSNA                           |0        |
|CSCTEXTRON-CESSNA                           |CSCTEXTRON-CESSNA                           |0        |
|ERNST AND YOUNG                             |ERNST AND YOUNG                     

In [57]:
result.where(F.col('endcustomerlinefixed_data')=='NAVAL DISTRICT WASHINGTON').show(10,False)

+-------------------------+-------------------------+-----------------------+------------------+---------+
|endcustomerlinefixed_data|party_name_savm_grpd     |sales_acct_id_savm_grpd|sales_acct_id_data|predicted|
+-------------------------+-------------------------+-----------------------+------------------+---------+
|NAVAL DISTRICT WASHINGTON|NAVAL DISTRICT WASHINGTON|2.17901541E8           |2.03710101E8      |0        |
+-------------------------+-------------------------+-----------------------+------------------+---------+



In [58]:
contest_savm.where(F.col('party_name')=='NAVAL DISTRICT WASHINGTON').select('sales_acct_id').show(10,False)

+-------------+
|sales_acct_id|
+-------------+
|2.17901632E8 |
|2.17901498E8 |
|2.17901541E8 |
|2.17901591E8 |
|2.17901572E8 |
|2.17901498E8 |
|2.17901632E8 |
|2.17901541E8 |
|2.17901572E8 |
|2.17901591E8 |
+-------------+
only showing top 10 rows



In [149]:
# Write a function which checks the entire name column and returns an array for of matching candidates
from math import *
def jaccard_similarity(x,y):
    if (isinstance(x,str) and isinstance(y,str)):
        intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
        union_cardinality = len(set.union(*[set(x), set(y)]))
        return intersection_cardinality/float(union_cardinality)
    else: return 0

In [73]:
result = contest_savm.map(lambda x : similarity_check(x.party_name)).collect()

In [72]:
def similarity_check(name):
    if (jaccard_similarity('NAVAL DISTRICT WASHINGTON',name))>0.8:
        return name 

In [74]:
result

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [40]:
#result_unq.rdd.map(lambda r : r(0)).collect()

In [21]:
contest_data_names = contest_data_unq.rdd.map(lambda r : r(0)).collect()

In [24]:
result_names = result_unq.rdd.map(lambda r : r(0)).collect()

In [35]:
result_new = [x for x in result_names if x in contest_data_names]

In [38]:
#contest_data_sample.take(2)

In [224]:
result.where(F.col('predicted')=='true').count()

0

In [31]:
#take the most occuring id
#grpd.orderBy("endcustomerlinefixed_data",desc("count")).show(5)
#grpd.orderBy(["endcustomerlinefixed_data", "count"], ascending=[0, 0]).dropDuplicates(['endcustomerlinefixed_data']).show(100,False)

In [ ]:
#join back with original dataset, and check accuracy

In [32]:
#grpd.sort("count", ascending=False).show(100,False)

In [33]:
#grpd.sort(grpd.count.desc()).collect()
#grpd.sort("count", ascending=False).collect()

In [7]:
## Reproducing R example given in Business Context Document
selected_line = contest_data.where(F.col('sales_order_line_key')== 240314891)

In [9]:
selected_line.take(5)

[Row(endcustomerlinefixed=u'CVS CAREMARK CORPORATION', pppk=12551412, fppk=6520976, decision_date_time=u'2015-07-31 14:48:15.0', days_from_create_to_final_decision=1.083333, end_customer_party_ssot_party_id_int_sav_party_id=10441, prior_party_ssot_party_id_int_sav_party_id=10441, final_party_ssot_party_id_int_sav_party_id=976308, type=u'Reviewed_and_modified', priorpartyname=u'CVS CAREMARK CORPORATION', finalpartyname=u'CVS PHARMACY, INC.', sales_order_key=24163958, sales_order_line_key=240314891, so_number_int=100973793, end_customer_key_line_level=41488655, end_customer_key_order_level=41488655, dd_end_customer_type_order_level=u'ACCOUNT_LOCATION                                  ', purchase_order_type_code=u'Resale', purchase_order_number=u'59799', order_datetime=u'2015-07-30 12:20:56.0', oracle_book_datetime=u'2015-07-30 12:28:03.0', transactional_currency_code=u'USD            ', customer_service_rep_name=u'null', cust_svc_rep_team_name=u'null', conversion_type_code=u'Corporate', o

In [10]:
contest_cr.where(F.col('party_id')== 10441).take(5)

[Row(party_id=10441.0, parent_party_id=None, party_name=u'CVS CAREMARK CORPORATION', node_type=u'HQ', address1=u'1 CVS DR', address2=None, address3=None, address4=None, city=u'WOONSOCKET', county=u'PROVIDENCE', state=u'RI', province=None, postal_code=u'02895', postal_code_extn=u'6146', country_code=u'US', street_name=u'CVS', street_number=u'1', street_direction=None, street_type=u'DR', geo_valid_status=u'GEO_VALID', completenes_status=u'COMPLETE', cleansed_status=u'CLEANSED', start_date=u'2002-11-11 22:10:07.0', end_date=None, program_id=None, request_id=None, created_by=1116.0, last_updated_by=332195.0, creation_date=u'2002-11-11 22:10:07.0', last_update_date=u'2014-05-09 17:22:42.0', certified_date=u'2014-05-09 17:22:42.0')]

In [11]:
contest_savm.where(F.col('party_id')== 10441).take(5)

[Row(party_id=10441.0, parent_party_id=None, party_name=u'CVS CAREMARK CORPORATION', node_type=u'HQ', address1=u'1 CVS DR', address2=None, address3=None, address4=None, city=u'WOONSOCKET', county=u'PROVIDENCE', state=u'RI', province=None, postal_code=u'02895', postal_code_extn=u'6146', country_code=u'US', street_name=u'CVS', street_number=u'1', street_direction=None, street_type=u'DR', geo_valid_status=u'GEO_VALID', completenes_status=u'COMPLETE', cleansed_status=u'CLEANSED', start_date=u'2002-11-11 22:10:07.0', end_date=None, program_id=None, request_id=None, created_by=1116.0, last_updated_by=332195.0, creation_date=u'2002-11-11 22:10:07.0', last_update_date=u'2014-05-09 17:22:42.0', certified_date=u'2014-05-09 17:22:42.0', site_expl_id=447210597.0, conflict_batch_id=83250456.0, sa_member_id=1173193.0, parent_sa_member_id=28146.0, party_level=2.0, link_party_id=203667683.0, link_party_type=u'TEMPLATE', split_pct=100.0, sales_acct_id=205775271.0, operation_type=u'INSERT', account_type

In [12]:
contest_savm.where(F.col('party_id')== 976308).take(5)

[Row(party_id=976308.0, parent_party_id=2358399.0, party_name=u'CVS PHARMACY, INC.', node_type=u'BR', address1=u'2154 FORT WORTH AVE', address2=None, address3=None, address4=None, city=u'DALLAS', county=u'DALLAS', state=u'TX', province=None, postal_code=u'75211', postal_code_extn=u'1811', country_code=u'US', street_name=u'FORT WORTH', street_number=u'2154', street_direction=None, street_type=u'AVE', geo_valid_status=u'GEO_VALID', completenes_status=u'COMPLETE', cleansed_status=u'CLEANSED', start_date=u'2002-11-12 05:18:45.0', end_date=u'4712-12-31 00:00:01.0', program_id=46063.0, request_id=103145420.0, created_by=1116.0, last_updated_by=316142.0, creation_date=u'2002-11-12 05:18:45.0', last_update_date=u'2014-02-14 22:45:30.0', certified_date=u'2014-02-14 22:45:30.0', site_expl_id=441093704.0, conflict_batch_id=83250358.0, sa_member_id=1173193.0, parent_sa_member_id=28164.0, party_level=3.0, link_party_id=203667683.0, link_party_type=u'TEMPLATE', split_pct=100.0, sales_acct_id=2057752

In [146]:
## Inspecting the curious case of 'BECHTEL MARINE PROPULSION CORPORATION'
contest_data.where(F.col('end_customer_party_ssot_party_id_int_sav_party_id')== 7689590).take(1)#.select('endcustomerlinefixed','end_customer_party_ssot_party_id_int_sav_party_id','prior_party_ssot_party_id_int_sav_party_id','sales_acct_id','decision_date_time').take(10)

[]

Corner Cases:
    265607419,150614328, 67510420,237234969,4448764,2636291,247235349,280675961

In [46]:
test = contest_data_sample.sample(False,0.002).select('end_customer_party_ssot_party_id_int_sav_party_id','sales_acct_id','decision_date_time').take(1) #Randomly picking an entry
test[0]

Row(end_customer_party_ssot_party_id_int_sav_party_id=266066405, sales_acct_id=203700596.0, decision_date_time=u'2016-07-21 14:43:56.0')

In [47]:
test_case = contest_cr.where(F.col('party_id')== test[0][0])
test_case.take(1)
#contest_cr.where(F.col('party_id')== 265607419).take(2)#.select('party_id','party_name','address1','city','county','state','postal_code','start_date').take(5)

[Row(party_id=266066405.0, parent_party_id=188095052.0, party_name=u'ONE INC SYSTEMS', node_type=u'BR', address1=u'620 COOLIDGE DR', address2=None, address3=None, address4=None, city=u'FOLSOM', county=u'SACRAMENTO', state=u'CA', province=None, postal_code=u'95630', postal_code_extn=None, country_code=u'US', street_name=u'COOLIDGE', street_number=u'620', street_direction=None, street_type=u'DR', geo_valid_status=u'GEO_VALID', completenes_status=u'COMPLETE', cleansed_status=u'CLEANSED', start_date=u'2016-01-05 14:13:45.0', end_date=u'4712-12-31 00:00:01.0', program_id=None, request_id=None, created_by=307958.0, last_updated_by=307958.0, creation_date=u'2016-01-05 14:13:45.0', last_update_date=u'2016-01-05 14:13:45.0', certified_date=u'2016-02-24 15:33:20.0')]

In [49]:
all_acct_ids = contest_savm.where\
((F.col('postal_code').rlike("^95630$"))  & (F.col('start_date')>=test[0][2]))#.take(50)
actual_acct_id = all_acct_ids.where(F.col('sales_acct_id')==test[0][1])
print actual_acct_id.count(),'/',all_acct_ids.select('sales_acct_id').count()
print 'Distinct sales_acct_ids: ',all_acct_ids.select('sales_acct_id').distinct().count()

0 / 43
Distinct sales_acct_ids:  36


In [35]:
freq = all_acct_ids.stat.freqItems(['sales_acct_id'], 0.25)
freq.collect()[0]

Row(sales_acct_id_freqItems=[203777527.0, 203686553.0, 253850764.0])

In [36]:
#See correlation with actual: ARMY RESEARCH LAB
print 'See correlation with actual: ',test_case.select('party_name').take(1)
actual_acct_id.select('party_name').distinct().show(10,False)

See correlation with actual:  [Row(party_name=u'VOLKSWAGEN')]
+----------+
|party_name|
+----------+
|SEAT      |
+----------+



In [37]:
#See correlation with actual: SHIPPING & RECEIVING, BLDG 1856
print 'See correlation with actual: ',test_case.select('address1').take(1)
actual_acct_id.select('address1').distinct().show(10,False)

See correlation with actual:  [Row(address1=u"CALLE L'ACER 30")]
+--------------------+
|address1            |
+--------------------+
|PASEO ZONA FRANCA 51|
|PASEO ZONA FRANCA 55|
+--------------------+



In [38]:
#See correlation with actual: CVS
print 'See correlation with actual: ',test_case.select('street_name').take(1)
actual_acct_id.select('street_name').distinct().show(10,False)

See correlation with actual:  [Row(street_name=u"L'ACER")]
+------------------------+
|street_name             |
+------------------------+
|PASEO ZONA FRANCA 51 185|
|ZONA FRANCA             |
+------------------------+



**ALGORITHM:**
1. Join with contest_cr table on end_customer_party_ssot_party_id_int_sav_party_id == party_id
2. For each entry in the above join, create a new filtered dataframe by filtering down on start_date > decision_date_time. Further filter on postal_code first see if there is perfect match, if not then keep emitting last one digit till there is atleast one match.
3. In the above filtered dataframe, fuzzy matching on party_name, if any result more than 0.9, send the best result.
4. If result of step 3 isn't more than 0.9 for any record, then repeat step 3 for address1, and street_name
5. In case of no match, send the result with max score.

Filter out non-US addresses, Zip code starting with 22... indicating area around capital (govt instiutions)

Join on name, address1, street_name, see match percentage. If greater than 90%, filter out those.

In [4]:
# Write a function which checks the entire name column and returns an array for of matching candidates
from math import *
def jaccard_similarity(x,y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    if float(union_cardinality) != 0:
        return intersection_cardinality/float(union_cardinality)
    else:
        return 0

In [8]:
smpl = contest_data_sample.select('end_customer_party_ssot_party_id_int_sav_party_id','sales_acct_id','decision_date_time') #Randomly picking few enteries

In [9]:
smpl.count()

1057

In [10]:
smpl_join = smpl.join(contest_cr, F.col('end_customer_party_ssot_party_id_int_sav_party_id')==F.col('party_id'))#.select('party_id','decision_date_time','party_name','postal_code','address1','street_name','sales_acct_id').cache()

In [11]:
smpl.select('end_customer_party_ssot_party_id_int_sav_party_id').distinct().count()

619

In [12]:
smpl_join_r = smpl_join.select(*(F.col(x).alias(x + '_orig') for x in smpl_join.columns))
join_a = smpl_join_r.join(contest_savm,F.col('postal_code_orig')==F.col('postal_code')).where(F.col('decision_date_time_orig')<=F.col('start_date'))

In [13]:
#join_a.cache()
join_a.count()

189701

In [14]:
join_a = join_a.na.fill('a',["party_name_orig","party_name","address1_orig","address1","street_name_orig","street_name"])

In [15]:
jackard_udf = F.udf(jaccard_similarity,types.DoubleType())
join_b = join_a.withColumn("name_similairity", jackard_udf(join_a["party_name_orig"],join_a["party_name"]))
join_b = join_b.withColumn("address_similairity", jackard_udf(join_a["address1_orig"],join_a["address1"]))
join_b = join_b.withColumn("st_name_similairity", jackard_udf(join_a["street_name_orig"],join_a["street_name"]))

In [16]:
def find_prediction(savm_id,data_id):
    return 1 if savm_id==data_id else 0

find_prediction_udf = F.udf(find_prediction,types.IntegerType())
join_b = join_b.withColumn("predicted", find_prediction_udf(join_b["sales_acct_id_orig"],join_b["sales_acct_id"]))#.select(['endcustomerlinefixed_data','predicted'])

In [43]:
join_b.select('party_id_orig','name_similairity','address_similairity','st_name_similairity','predicted').show(5)

+-------------+-------------------+-------------------+-------------------+---------+
|party_id_orig|   name_similairity|address_similairity|st_name_similairity|predicted|
+-------------+-------------------+-------------------+-------------------+---------+
| 2.18812761E8|0.47619047619047616| 0.9166666666666666|                1.0|        0|
| 2.18812761E8|  0.631578947368421| 0.3333333333333333|                0.0|        0|
| 2.18812761E8|0.47619047619047616| 0.9166666666666666|                1.0|        0|
| 2.18812761E8|0.47619047619047616| 0.9166666666666666|                1.0|        0|
| 2.18812761E8| 0.5555555555555556| 0.3333333333333333| 0.1111111111111111|        0|
+-------------+-------------------+-------------------+-------------------+---------+
only showing top 5 rows



In [41]:
#jackard_threshold = 0.95
#join_c = join_b.where((F.col('name_similairity')>jackard_threshold) | (F.col('address_similairity')>jackard_threshold)| (F.col('st_name_similairity')>jackard_threshold))
#join_c.select('name_similairity','address_similairity','st_name_similairity','predicted').show(5)

In [255]:
c_pd = join_c.toPandas()

In [256]:
X = c_pd[['name_similairity','address_similairity','st_name_similairity']]
y = c_pd['predicted']

In [257]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [258]:
logmodel.coef_

array([[ 4.37695095,  1.28740361, -1.0817297 ]])

In [273]:
#c_pd.dtypes

In [26]:
#[f.dataType for f in join_c.schema.fields]
#join_c.cache()

In [21]:
def aggregated_prediction(name,adr,st):
    return (4.3*name + 1.2*adr)

aggregated_prediction_udf = F.udf(aggregated_prediction,types.DoubleType())
join_c = join_b.withColumn("aggregated_prediction", aggregated_prediction_udf(join_b['name_similairity'],join_b['address_similairity'],join_b['st_name_similairity']))#.select(['endcustomerlinefixed_data','predicted'])

In [49]:
join_c.select('party_id_orig','aggregated_prediction','predicted').show(50)

+-------------+---------------------+---------+
|party_id_orig|aggregated_prediction|predicted|
+-------------+---------------------+---------+
| 2.18812761E8|   3.0666666666666664|        0|
| 2.18812761E8|   2.3224561403508774|        0|
| 2.18812761E8|     3.11578947368421|        0|
| 2.18812761E8|   2.7888888888888888|        0|
| 2.18812761E8|   3.2895833333333333|        0|
| 2.18812761E8|    3.147619047619047|        0|
| 2.18812761E8|   3.0221052631578944|        0|
| 2.18812761E8|    3.147619047619047|        0|
| 2.18812761E8|    2.223529411764706|        0|
| 2.18812761E8|    3.147619047619047|        0|
| 2.18812761E8|    3.147619047619047|        0|
| 2.18812761E8|    3.147619047619047|        0|
| 2.18812761E8|    3.147619047619047|        0|
| 2.18812761E8|   2.8987499999999997|        0|
| 2.18812761E8|    3.147619047619047|        0|
| 2.18812761E8|   3.3352941176470585|        0|
| 2.18812761E8|   3.3562499999999997|        0|
| 2.18812761E8|    3.147619047619047|   

In [62]:
#join_c.sort(F.col("party_id_orig").desc()).sort(F.col("part").desc()))
join_c.orderBy(F.desc("party_id_orig"),F.desc("aggregated_prediction")).select('party_id_orig','aggregated_prediction','predicted').show(500)

+-------------+---------------------+---------+
|party_id_orig|aggregated_prediction|predicted|
+-------------+---------------------+---------+
|  3.0075236E8|                  5.5|        1|
|  3.0075236E8|              5.23125|        1|
|  3.0075236E8|    5.213333333333334|        1|
|  3.0075236E8|   5.0636363636363635|        1|
|  3.0075236E8|   4.8999999999999995|        1|
|  3.0075236E8|    4.776969696969697|        1|
|  3.0075236E8|    4.776969696969697|        1|
|  3.0075236E8|    4.776969696969697|        1|
|  3.0075236E8|    4.776969696969697|        1|
|  3.0075236E8|    4.776969696969697|        1|
|  3.0075236E8|    4.776969696969697|        1|
|  3.0075236E8|    4.776969696969697|        1|
|  3.0075236E8|    4.776969696969697|        1|
|  3.0075236E8|    4.776969696969697|        1|
|  3.0075236E8|    4.776969696969697|        1|
|  3.0075236E8|    4.776969696969697|        1|
|  3.0075236E8|    4.776969696969697|        1|
|  3.0075236E8|    4.776969696969697|   

In [65]:
join_c.registerTempTable("x")
predictions_table = sqlContext.sql("SELECT party_id_orig,aggregated_prediction,predicted  FROM( SELECT *, ROW_NUMBER()OVER(PARTITION BY party_id_orig ORDER BY aggregated_prediction DESC) rn FROM x) y WHERE rn = 1").cache()

In [66]:
predictions_table.count()

569

In [68]:
predictions_table.where(F.col('predicted')==1).count()

211

In [48]:
join_c.groupby("predicted").agg(F.avg("aggregated_prediction")).show()

+---------+--------------------------+
|predicted|avg(aggregated_prediction)|
+---------+--------------------------+
|        0|        2.2483320566057095|
|        1|        3.8363411453001794|
+---------+--------------------------+



In [54]:
#join_d = join_c.groupby("party_id_orig",'predicted').agg(F.max("aggregated_prediction"))

In [55]:
#join_d.show(5)

In [51]:
join_d.count()

862

In [ ]:
join_d.where(F.col(''))

In [56]:
#join_d=join_c.groupby('party_id_orig').agg({"name_similairity": "max"})
join_d = join_c.groupby("party_id_orig").agg(F.max("aggregated_prediction"))
join_d = join_d.withColumn("highest_match", F.lit(1))


In [35]:
join_d.count()

569

In [57]:
join_d.show(5)

+-------------+--------------------------+-------------+
|party_id_orig|max(aggregated_prediction)|highest_match|
+-------------+--------------------------+-------------+
|  2.4913913E8|                      5.07|            1|
| 2.17583603E8|                   4.69375|            1|
| 1.84732648E8|        3.7549019607843133|            1|
| 2.29404851E8|         3.010989010989011|            1|
| 2.37234395E8|         3.403030303030303|            1|
+-------------+--------------------------+-------------+
only showing top 5 rows



In [58]:
join_e = join_d.join(join_b,'party_id_orig')['party_id_orig',"aggregated_prediction",'highest_match','predicted']

AnalysisException: u"cannot resolve 'aggregated_prediction' given input columns: [account_sub_type, creation_date_orig, parent_party_id, street_name, max(aggregated_prediction), site_expl_id, postal_code_extn, address3_orig, account_type, operation_type, party_name, state_orig, program_id_orig, country_code_orig, geo_valid_status_orig, name_similairity, link_party_id, address4_orig, certified_date, sa_member_id, st_name_similairity, street_type, postal_code_orig, creation_date, split_pct, address2, sales_acct_id, program_id, certified_date_orig, street_number_orig, postal_code, party_name_orig, start_date, created_by_orig, county_orig, end_date, created_by, country_code, last_update_date, party_id_orig, party_id, sales_acct_id_orig, predicted, highest_match, street_type_orig, province, address2_orig, request_id_orig, province_orig, link_party_type, street_name_orig, address_similairity, street_direction_orig, end_customer_party_ssot_party_id_int_sav_party_id_orig, address1_orig, node_t

In [37]:
join_e.count()

189701

In [38]:
join_e.show(5)

+-------------+-------------+---------+
|party_id_orig|highest_match|predicted|
+-------------+-------------+---------+
| 1.84732648E8|            1|        0|
| 1.84732648E8|            1|        0|
| 1.84732648E8|            1|        0|
| 1.84732648E8|            1|        0|
| 1.84732648E8|            1|        0|
+-------------+-------------+---------+
only showing top 5 rows



In [39]:
join_e.select('party_Id_orig').distinct().count()

569

In [40]:
join_e.where(F.col('highest_match')==1).count()

189701

In [271]:
j_pd = join_d.toPandas()

In [272]:
c_pd.join(j_pd,'party_id_orig')['party_id_orig','max(aggregated_prediction)','predicted']

ValueError: columns overlap but no suffix specified: Index([u'party_id_orig'], dtype='object')

In [113]:
print smpl.select('end_customer_party_ssot_party_id_int_sav_party_id').distinct().count()
print smpl_join.select('party_id').distinct().count()
print join_a.select('party_id_orig').distinct().count()
print join_b.select('party_id_orig').distinct().count()
print join_c.select('party_id_orig').distinct().count()
print join_d.select('party_id_orig').distinct().count()

619
618
569
569
334
334


In [115]:
join_c.count()

13844

In [365]:
def predict_acct_id(id,date,name,zip,address,st_name):
    all_acct_ids = contest_savm.where((F.col('postal_code')==zip)  & (F.col('start_date')>=date))
    if (jaccard_similarity('name',name))>0.8
    return all_acct_ids.count()

In [366]:
predict_acct_id(2640238,'2015-12-09 15:44:08.0','CITY OF CEDAR PARK',78613,'450 CYPRESS CREEK RD - 120','CYPRESS CREEK')

70

In [357]:
predict_acct_id_udf = F.udf(predict_acct_id,types.DoubleType())
#url_cleansing_udf = F.udf(cleanse_url,types.StringType())
#smpl_new = smpl_join.withColumn("predicted_acct", predict_acct_id_udf(smpl_join["party_id"],smpl_join["decision_date_time"],smpl_join["party_name"],smpl_join["postal_code"],smpl_join["address1"],smpl_join["street_name"]))

Py4JError: An error occurred while calling o49.__getnewargs__. Trace:
py4j.Py4JException: Method __getnewargs__([]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:335)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:344)
	at py4j.Gateway.invoke(Gateway.java:252)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:133)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:209)
	at java.lang.Thread.run(Thread.java:745)



In [362]:
smpl_new = smpl_join.select("party_id").rdd\
  .map(lambda x: predict_acct_id(x[0],x[1],x[2],x[3],x[4],x[5]))\
  .distinct()

Py4JError: An error occurred while calling o49.__getnewargs__. Trace:
py4j.Py4JException: Method __getnewargs__([]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:335)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:344)
	at py4j.Gateway.invoke(Gateway.java:252)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:133)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:209)
	at java.lang.Thread.run(Thread.java:745)



In [358]:
def predict_acct_id(id):
    #all_acct_ids = contest_savm.where((F.col('postal_code')==zip)  & (F.col('start_date')>=date))
    return id+2

In [360]:
smpl_new.take(2)

[222536522.0, 122823413.0]

In [338]:
typ = [f.dataType for f in smpl_join.schema.fields]

In [339]:
typ

[DoubleType,
 StringType,
 StringType,
 StringType,
 StringType,
 StringType,
 DoubleType]

In [119]:
contest_savm.where\
((F.col('postal_code').rlike("^83401$"))  & (F.col('start_date')>'2012-09-01 03:04:11.0'))\
.select('party_id','sales_acct_id','party_name','address1','city','county','state','postal_code','street_name','start_date')\
.take(5)

[Row(party_id=219550442.0, sales_acct_id=203816119.0, party_name=u'DIRECTV LLC', address1=u'1245 HOLLIPARK DR', city=u'IDAHO FALLS', county=u'BONNEVILLE', state=u'ID', postal_code=u'83401', street_name=u'HOLLIPARK', start_date=u'2014-05-10 11:33:58.0'),
 Row(party_id=221913104.0, sales_acct_id=203802231.0, party_name=u'US  CELLULAR CORPORATION', address1=u'570 S WOODRUFF AVE', city=u'IDAHO FALLS', county=u'BONNEVILLE', state=u'ID', postal_code=u'83401', street_name=u'WOODRUFF', start_date=u'2014-07-11 14:08:35.0'),
 Row(party_id=221701713.0, sales_acct_id=203711563.0, party_name=u'WALGREEN COMPANY', address1=u'1725 1ST ST', city=u'IDAHO FALLS', county=u'BONNEVILLE', state=u'ID', postal_code=u'83401', street_name=u'1ST', start_date=u'2014-07-10 21:48:23.0'),
 Row(party_id=278120074.0, sales_acct_id=203688931.0, party_name=u'INL BATTELLE ENERGY ALLIANCE', address1=u'1765 N YELLOWSTONE HWY', city=u'IDAHO FALLS', county=u'BONNEVILLE', state=u'ID', postal_code=u'83401', street_name=u'PO BOX

In [ ]:
# Right acct_id = 203688931.0

In [52]:
pd_builder = builder.toPandas()

In [35]:
builder.select('endcustomerlinefixed_data').distinct().show(10,False)

+--------------------------+
|endcustomerlinefixed_data |
+--------------------------+
|INTEGRIS HEALTH           |
|CNP                       |
|AAFES PROCUREMENT IT-Z    |
|DILLARD'S, INC.           |
|DILLARD STORE SERVICES INC|
|Dillard's, Inc.           |
|HERC RENTALS              |
|AFRL DET 15               |
|HERTZ                     |
|DILLARD'S INC             |
+--------------------------+



In [49]:
builder.where(F.col('endcustomerlinefixed_data')=="HERC RENTALS").select(builder.endcustomerlinefixed_data,builder.decision_date_time_data,builder.sales_acct_id_data,builder.sales_acct_id_savm).count()

0

In [74]:
pd_austin = pd_builder[pd_builder['endcustomerlinefixed_data']=="DILLARD'S, INC."]

In [75]:
len(pd_austin)

11390

In [73]:
pd_builder.sample()

,endcustomerlinefixed_data,pppk_data,fppk_data,decision_date_time_data,days_from_create_to_final_decision_data,end_customer_party_ssot_party_id_int_sav_party_id_data,prior_party_ssot_party_id_int_sav_party_id_data,final_party_ssot_party_id_int_sav_party_id_data,type_data,priorpartyname_data,finalpartyname_data,sales_order_key_data,sales_order_line_key_data,so_number_int_data,end_customer_key_line_level_data,end_customer_key_order_level_data,dd_end_customer_type_order_level_data,purchase_order_type_code_data,purchase_order_number_data,order_datetime_data,oracle_book_datetime_data,transactional_currency_code_data,customer_service_rep_name_data,cust_svc_rep_team_name_data,conversion_type_code_data,original_system_reference_code_data,submitted_on_web_by_name_data,deal_id_data,price_list_name_data,so_source_name_data,order_type_name_data,so_src_crt_datetime_data,sales_channel_code_data,sales_channel_source_type_data,sales_order_category_type_data,ship_to_customer_key_data,bill_to_customer_key_data,sold_to_customer_key_data,ss_code_data,end_ssp_date_data,ru_bk_orig_sales_order_key_data,so_created_by_erp_user_name_data,so_modified_by_erp_user_name_data,sk_sales_order_header_id_int_data,sales_order_program_type_name_data,sales_acct_id_data,party_id_savm,parent_party_id_savm,party_name_savm,node_type_savm,address1_savm,address2_savm,address3_savm,address4_savm,city_savm,county_savm,state_savm,province_savm,postal_code_savm,postal_code_extn_savm,country_code_savm,street_name_savm,street_number_savm,street_direction_savm,street_type_savm,geo_valid_status_savm,completenes_status_savm,cleansed_status_savm,start_date_savm,end_date_savm,program_id_savm,request_id_savm,created_by_savm,last_updated_by_savm,creation_date_savm,last_update_date_savm,certified_date_savm,site_expl_id_savm,conflict_batch_id_savm,sa_member_id_savm,parent_sa_member_id_savm,party_level_savm,link_party_id_savm,link_party_type_savm,split_pct_savm,sales_acct_id_savm,operation_type_savm,account_type_savm,account_sub_type_savm
12062,"DILLARD'S, INC.",7765504,10980279,2015-06-11 09:18:12.0,36.958333,12847,12847,10344837,Reviewed_and_modified,"DILLARD'S, INC.",DILLARD'S PORTRAIT STUDIO,23650829,232963538,100548416,56790793,56790793.0,ACCOUNT_LOCATION,Resale,118029,2015-05-05 10:15:39.0,2015-05-05 10:19:21.0,USD,null,null,Corporate,63619968,sdupree2011,15372888,Global Price List - US,Commerce Workspace,Standard Cisco - US,2015-05-05 10:18:34.0,Unknown,ERP,ORDER,56790793.0,16150530.0,10192337.0,CG,3500-01-01 00:00:00.0,-9999.0,CCWADM,AMSHARM2,602039902.0,Standard,203684849.0,5352888.0,12847.0,"DILLARD'S, INC.",BR,4915 HELLER ST,None,None,None,LOUISVILLE,JEFFERSON,KY,None,40218,3481,US,HELLER,4915,None,ST,GEO_VALID,COMPLETE,CLEANSED,2002-11-12 18:15:53.0,4712-12-31 00:00:00.0,44449.0,42293842.0,1116.0,305153.0,2002-11-12 18:15:53.0,2010-03-18 10:00:44.0,2010-03-18 10:00:43.0,453412543.0,83961122.0,3243401.0,-1.0,2.0,12847.0,ORGANIZATION,100.0,203684849.0,INSERT,NAMED_ACCOUNT,DR


In [76]:
pd_austin_tr = pd_austin[['decision_date_time_data','start_date_savm','sales_acct_id_savm','sales_acct_id_data']]

In [34]:
#pd_builder.groupby(['endcustomerlinefixed_data','sales_acct_id_savm', 'sales_acct_id_data']).size()

In [35]:
#builder.take(1)

In [51]:
builder.select(builder.sales_acct_id_data).count()

57975

In [31]:
builder = contest_data.select(*(F.col(x).alias(x + '_cd') for x in contest_data.columns))
builder = contest_savm.join(builder, builder.sales_acct_id_cd == contest_savm.sales_acct_id, how = 'inner') # Exploding 68 times

In [32]:
builder.count()

683250

In [36]:
#builder = builder.select(*(F.col(x).alias(x + '_cd') for x in builder.columns))
#builder.columns

In [37]:
#builder = contest_so_end_customer.join(builder, builder.sales_order_key_cd_cd == contest_so_end_customer.sales_order_key, how='inner')
#builder = contest_end_customer.join(builder, 'end_customer_key', how='inner')

In [19]:
#if column is duplicated, drop the later column
#TODO optimize for only 1 select
def apply_function(df, fields, function):
    column_objects = []
    for column in df.columns:
        if column in fields:
            column_objects.append(function(column))
        else:
            column_objects.append(column)
    return df.select(column_objects)

max_date = pd.datetime.today()

#inputs to apply_function
def function_binarize(null_value):
    def inner(column):
        return F.when(F.col(column) == null_value, 0).otherwise(1).alias(column)
    return inner

def function_binarize_nulls():
    return lambda column : F.when(F.col(column).isNull(), 0).otherwise(1).alias(column)

def function_datetimeparse():
    def inner(column):
        def parse(d):
            if isinstance(d, str):
            #try:
                return(max_date - datetime.strptime(d, '%Y-%m-%d %H:%M:%S.0')).days
            return d
            #except ValueError:
            #    return 0
        return F.udf(parse, types.IntegerType())(F.col(column)).alias(column)
    return inner

def function_replace_empty_strings(value):
    return lambda column : F.when(F.length(F.col(column)) == 0, value).otherwise(F.col(column)).alias(column)

def function_replace_nulls(value):
    return lambda column : F.when(F.col(column).isNull(), value).otherwise(F.col(column)).alias(column)

def function_replace_nans(value):
    return lambda column : F.when(F.col(column) == np.nan, value).otherwise(F.col(column)).alias(column)

#because multidrop in pyspark is ugly
def drop_columns(df, columns):
    return df.select([c for c in df.columns if c not in columns])

class FrequencyThreshold:
    def __init__(self, df, columns):
        self.freq_items = df.freqItems(columns, 0.01)
        
    def freq_threshold(self, column):
        freq_items = self.freq_items.flatMap(lambda x : x[column+"_freqItems"]).collect()
        return F.when(F.col(column).isin(freq_items), F.col(column)).otherwise('OTHER').alias(column)
        

In [23]:
def drop_duplicate_columns(df):
    column_objects = []
    for col in df.columns:
        if col not in column_objects:
            column_objects.append(col)#stripping leading and trailing spaces
    return df.select(column_objects)

In [7]:
from itertools import islice
def check_skewness(df):
    sampled_rdd = df.sample(False,0.01).rdd.cache() # Taking just 1% sample
    l = sampled_rdd.mapPartitionsWithIndex(lambda x,it: [(x,sum(1 for _ in it))]).collect()
    max_part,min_part = max(l,key=lambda item:item[1]), min(l,key=lambda item:item[1])
    if max_part[1]/min_part[1] > 5: #if difference is greater than 5%
        print 'Partitions Skewed: Largest Partition',max_part,'Smallest Partition',min_part,'\nSample Content of the largest Partition: \n'
        print (sampled_rdd.mapPartitionsWithIndex(lambda i, it: islice(it, 0, 5) if i == max_part[0] else []).take(5))
    else:
        print 'No Skewness: Largest Partition',max_part,'Smallest Partition',min_part

# Name matching

In [12]:
#creating function to create dictionary:
import string
name_freq_table = {}
stop_words = ['GROUP','LLC','INC','LTD','LIMITED','MEDIA','COUNTY','LLP','&','AND','AGENCY','LP','CORPORATION','COMPANY','DEPT','DEPTT','USA','CORP','COMMUNICATIONS','AMERICA','NORTH AMERICA', 'SYSTEMS', 
'FORUM', 'LOGISTICS', 'DEPT', 'SERVICES', 'OF', 'THE', 'NA', 'BRANDS', 'STATE','CORP','ST', 'SELECT', 'INTERNATIONAL','COMMERCIAL','HOLDINGS','HOLDING','ENTERPRISE']

def name_freq(name1,name2):
    global name_freq_table
    words1 = treat_names(name1)
    words2 = treat_names(name2)
    for word1 in words1:
        for word2 in words2:
            if word1 != word2:
                key = (word1,word2)
                if key in name_freq_table:
                    name_freq_table[key] += 1
                else:
                    name_freq_table[key] = 1
                    
def treat_names(name):
    name = ''.join(ch for ch in name if ch not in string.punctuation)
    resultwords  = [word for word in name.split() if word not in stop_words]
    return resultwords

In [56]:
name_freq('HONDA OF AMERICA MFG INC','HONDA OF AMERICA NEW YORK')

In [57]:
name_freq_table

{('HONDA', 'NEW'): 2,
 ('HONDA', 'YORK'): 2,
 ('MFG', 'HONDA'): 2,
 ('MFG', 'NEW'): 2,
 ('MFG', 'YORK'): 2}

In [58]:
records =  dict(((k,name_freq_table[k]),12) for k in name_freq_table)
records

{(('HONDA', 'NEW'), 2): 12,
 (('HONDA', 'YORK'), 2): 12,
 (('MFG', 'HONDA'), 2): 12,
 (('MFG', 'NEW'), 2): 12,
 (('MFG', 'YORK'), 2): 12}

In [68]:
DF = sqlContext.createDataFrame(records,['name_pair','count']).withColumn("word1", F.col('name_pair'))#.withColumn("x4", lit(1))

In [69]:
DF.show()

+------------+-----+------------+
|   name_pair|count|       word1|
+------------+-----+------------+
| [MFG,HONDA]|    2| [MFG,HONDA]|
| [HONDA,NEW]|    2| [HONDA,NEW]|
|  [MFG,YORK]|    2|  [MFG,YORK]|
|   [MFG,NEW]|    2|   [MFG,NEW]|
|[HONDA,YORK]|    2|[HONDA,YORK]|
+------------+-----+------------+



In [102]:
df2 = pd.DataFrame({
    'E' : ["test","train","test","train"],
    'F' : 'foo',
    'D' : np.array([3] * 4,dtype='int32')
    })

In [103]:
df2

,D,E,F
0,3,test,foo
1,3,train,foo
2,3,test,foo
3,3,train,foo


In [104]:
format = lambda x: x['E']+x['F']
df2.apply(format,axis=1)

0     testfoo
1    trainfoo
2     testfoo
3    trainfoo
dtype: object

In [105]:
from scipy.sparse import coo_matrix

In [107]:
coo_matrix((df2['D'], (df2['D'], df2['F']))).toarray()

TypeError: cannot concatenate 'str' and 'int' objects

In [108]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [109]:
le.fit(df2['E'])

LabelEncoder()

In [111]:
df2['E'] =le.transform(df2['E'])

In [112]:
df2

,D,E,F
0,3,0,foo
1,3,1,foo
2,3,0,foo
3,3,1,foo
